# ML4NGP training school 2026

# Deep generative models for IDP structural ensembles 🧬

# Hands-on session

---

## From sequence to coarse-grained structural ensembles with idpGAN

Welcome!

In this session, we will explore **deep generative models** that we can use to obtain structural ensembles of intrinsically disordered proteins (IDPs).

## Generative modeling

We will first use **[idpGAN](https://www.nature.com/articles/s41467-023-36443-x)**, a first-generation generative model trained to learn the distribution of conformations of coarse-grained (CG) IDPs.

At the end of the notebook, we will analyze some ensembles generated with more recent ML ensemble generators.

## Ensemble analysis

Generating an ensemble is only the first step. Another key part is their **analysis**.

We will mainly use **[IDPEnsembleTools (IDPET)](https://onlinelibrary.wiley.com/doi/10.1002/pro.70427)**, a Python library designed specifically for analyzing IDP/IDR ensembles.

We will also perform some analyses manually using **[NumPy](https://numpy.org/)**.

## Outcomes

By the end of this notebook, you will be able to:

- ✅ Generate a structural ensemble from a protein sequence with idpGAN
- ✅ Understand how ensemble data is represented in data analysis pipeline (numpy arrays or PyTorch tensors)
- ✅ Visualize some ensemble properties with IDPET (e.g., radius of gyration distributions)
- ✅ Compare structural ensembles (e.g.: to simulation-based references)
- ✅ Critically evaluate the strengths and limitations of ensemble generators

# Part 1/6 - Install and setup the model

In [ ]:
#@title Install dependencies and define useful functions

#@markdown Run this cell to install all the required dependencies for idpGAN. The installation will take place in the Colab notebook in the cloud. The whole process will take some minutes.

#
# Installation part.
#

# Download from GitHub the idpgan library and data.
!if [ ! -f "main.zip" ]; then wget https://github.com/feiglab/idpgan/archive/refs/heads/main.zip; fi
!if [ ! -d "idpgan-main" ]; then unzip -u main.zip idpgan-main/data/* idpgan-main/idpgan/*; fi
!if [ ! -d "data" ]; then mv idpgan-main/data .; fi
!if [ ! -d "idpgan" ]; then mv idpgan-main/idpgan .; fi
# Install the python dependencies for analyzing and visualizing 3D conformations.
!pip install mdtraj
!pip install idpet
!pip install py3Dmol


#
# Import the libraries we need.
#

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import mdtraj
import py3Dmol
import torch

from idpgan.data import (
    parse_fasta_seq,
    one_to_three,
)
from idpgan.nn_models import load_netg_article


#
# Useful functions for later.
#

def to_trajectory_and_top(xyz, seq, name, data_path):
    top_path = os.path.join(data_path, f"{name}.pdb")
    traj_path = os.path.join(data_path, f"{name}.xtc")
    seq_to_cg_pdb(seq, out_fp=top_path)
    pdb = mdtraj.load(top_path)
    traj = mdtraj.Trajectory(xyz=xyz, topology=pdb.topology)
    traj.save(traj_path)
    return traj

def seq_to_cg_pdb(seq, out_fp=None):
    """
    Gets an amino acid sequence and returns a template
    CG PDB file.
    """
    pdb_lines = []
    for i, aa_i in enumerate(seq):
        res_idx = i + 1
        aa_i = one_to_three[aa_i]
        line_i = "ATOM{:>7} CA   {} A{:>4}       0.000   0.000   0.000  1.00  0.00\n".format(
                   str(res_idx), aa_i, str(res_idx))
        pdb_lines.append(line_i)
    pdb_content = "".join(pdb_lines)
    if out_fp is not None:
        with open(out_fp, "w") as o_fh:
            o_fh.write(pdb_content)
    return pdb_content

def get_ca_topology(sequence: str):
    topology = mdtraj.Topology()
    chain = topology.add_chain()
    for res in sequence:
        res_obj = topology.add_residue(one_to_three[res], chain)
        topology.add_atom("CA", mdtraj.core.topology.elem.carbon, res_obj)
    return topology

def xyz_to_traj(xyz, seq):
    top = get_ca_topology(seq)
    traj = mdtraj.Trajectory(xyz=xyz, topology=top)
    return traj


def extract_rgb_from_colormap(colormap_name, N):
    # Create a colormap object.
    colormap = plt.get_cmap(colormap_name)
    # Generate N equally spaced values between 0 and 1.
    points = np.linspace(0, 1, N)
    # Use the colormap to get colors for these points, converting to HTML format.
    colors = [colormap(point)[:3] for point in points] # Extract RGB, ignore A.
    html_colors = ['#' + ''.join([f'{int(val*255):02x}' for val in color]) for color in colors]
    return html_colors

def show_structure(
        snapshot, representation='aa', cmap_name='viridis', max_structures=100
    ):
    pdb_fn = f'.temp.{time.time()}.pdb'
    vtraj = mdtraj.Trajectory(
        # max_structures only takes effect if we have more than 100 conformations.
        xyz=snapshot.xyz[:max_structures], topology=snapshot.topology
    )
    vtraj.superpose(vtraj, 0)
    vtraj.save(pdb_fn)
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if len(vtraj) == 1:
        with open(pdb_fn, 'r') as i_fh:
            view.addModel(i_fh.read(), 'pdb')
    else:
        with open(pdb_fn, 'r') as i_fh:
            view.addModelsAsFrames("".join([x for x in i_fh]))
    os.remove(pdb_fn)

    if representation == 'aa':
        view.setStyle({'cartoon': {'color': 'spectrum'},
                       'stick': {'color': 'spectrum'}})
    elif representation == 'cg':
        n_residues = snapshot.xyz.shape[1]
        colors = extract_rgb_from_colormap(cmap_name, n_residues)
        for i in range(n_residues):
            view.setStyle({'serial': str(i+1)},
                          {'sphere': {'color': colors[i],
                                      'radius': 1.9}})
    else:
        raise NotImplementedError(representation)

    view.zoomTo()
    if len(vtraj) > 1:
        view.animate({'loop': "forward", 'reps': 1, 'interval':1000})
    return view


#
# Setup some files.
#

temp_data_path = "data"

for seq_name in ("polyala", "protac"):
    _seq = parse_fasta_seq(os.path.join(temp_data_path, f"{seq_name}.fasta"))
    seq_to_cg_pdb(
        _seq, os.path.join(temp_data_path, f"{seq_name}.pdb")
    )
    _traj = mdtraj.Trajectory(
        xyz=np.load(
            os.path.join(temp_data_path, f"{seq_name}.npy")
        ),
        topology=mdtraj.load(
            os.path.join(temp_data_path, f"{seq_name}.pdb")
        ).topology
    )
    _traj[0:1000].save(os.path.join(temp_data_path, f"{seq_name}.xtc"))

## IdpGAN discussion

To understand the strengths and limitations of any AI model, you must know how it was trained. Before using a method, reading the original paper is always a good idea.

Let’s briefly recap the key facts about **idpGAN**.

### Model input/output
**Input:** a protein sequence  
**Output:** a 3D structural ensemble of a single IDP chain.
  - The number of conformations (`num_conformations`) is chosen by you.  
  - Each residue is represented by one bead (coarse-grained representation).


### Type of deep generative model
idpGAN is a **[Generative Adversarial Network (GAN)](https://en.wikipedia.org/wiki/Generative_adversarial_network)**.

GANs were popular in early generative modeling, but are less common today for problems due to training instability and poor performance on complex datasets.

### Training data

IdpGAN was trained on **coarse-grained (CG) molecular dynamics (MD)** simulations of IDPs.  
Their sequences were taken from **[DisProt](https://disprot.org)**.

The simulations were performed using **[COCOMO-1](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.2c00856)**.

- COCOMO-1 is a one-bead-per-residue CG model.
- It is conceptually similar to CALVADOS2 (which you already know). Both aim to reproduce some ensemble properties of IDPs.

You can think of idpGAN as a **fast emulator of COCOMO-1 simulations**.

❓**Question**❓: since idpGAN was trained on COCOMO-1 simulations:
* What ensemble properties should it reproduce well (think about CALVADOS)?
* What properties might it fail to capture?

## Load the idpGAN model

Loading the machine learning model (e.g.: loading the weights of its neural net) is typically the first step in using them.

In [ ]:
#
# Setup the model.
#

# Set the data directory of the idpGAN Git repository. Do not change if
# running on Colab.
data_path = "data"

# Setup the device in which we want to run the PyTorch models.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("- Using PyTorch device: '%s'" % device)

# Load the generator network that was used in the idpGAN paper.
netg = load_netg_article(model_fp=os.path.join(data_path, "generator.pt"),
                         device=device)
print("- IdpGAN generator loaded")

# Part 2/6 - Generate a first ensemble

Let's generate an IDP ensemble. We will use the [C-terminal part of human prothymosin α](https://www.pnas.org/doi/10.1073/pnas.1001743107) (here named: `protac`, tough it is totally unrelated to [PROTACS](https://en.wikipedia.org/wiki/Proteolysis_targeting_chimera)), which is an intrinsically disordered region (IDR). This sequence was in the test set of idpGAN.

## Generation

Define the sequence.

In [ ]:
# Sequence.
seq = "CEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDC"
seq_name = "protac"
print("sequence length:", len(seq))

# Number of conformations to generate. 1000 is already enough in many applications.
num_conformations = 1000

Actually generate the ensemble.

In [ ]:
%%time
xyz_gen = netg.predict_idp(n_samples=num_conformations, aa_seq=seq,
                           device=device).cpu().numpy()
print("- the generated xyz is an array of shape:", xyz_gen.shape)

Congrats! You have generated an IDP ensemble with ML. How much time did it take (see: `Wall time` in the cell above)?

❓**Question**❓:
  - The output above is an array with a shape like `(num_conformations, 55, 3)`? What is this array representing?

**Note**: this is a common way of representing structural ensemble data in computational biology and deep learning.

## Analysis via IDPET

Now, let's analyze the ensemble. The output of idpGAN is a [numpy](https://numpy.org) array. We need to convert it into a topology (PDB) file and a trajectory file first.

**Optional note: what are topology and trajectory files?** This combination is one of the most common ways of storing ensemble and Molecular Dynamics data.
- **Trajectory file**: a binary file containing all the xyz coordinates of the conformations in your system. It often does not store information about atom types (e.g.: carbon, oxygen, etc...).
- **Topology file**: a molecular structure file (here a PDB file) containing the type of atoms of your system. In some tools (idpGAN here and BioEmu) the PDB file also contains the coordinates of the first conformation in the ensemble (this is arbitrary).

Once we have that we can load in IDPET. Use the cell below to do it.

In [ ]:
# Run this to convert to a topology and PDB files that will be saved on the cloud.
to_trajectory_and_top(xyz_gen, seq, f"idpgan.{seq_name}", data_path)

Now let's load the ensemble in IDPET.

In [ ]:
from idpet.ensemble import Ensemble
from idpet.ensemble_analysis import EnsembleAnalysis
from idpet.visualization import Visualization


# Load the ensemble data.
ensemble = Ensemble(
    code=seq_name,
    data_path=os.path.join(data_path, f"idpgan.{seq_name}.xtc"),
    top_path=os.path.join(data_path, f"idpgan.{seq_name}.pdb")
)

analysis = EnsembleAnalysis([ensemble])
analysis.load_trajectories()
# Make the visualization object for visualizing ensemble features.
vis = Visualization(analysis=analysis)

How do we analyze an ensemble? The analysis of structural ensembles is an art/science in its own! It depends on the type of data (small vs big proteins, globular vs IDP, MD trajectories vs generic ensemble data).

For single chain IDPs, we typically begin by analyzing their [radius of gyration](https://en.wikipedia.org/wiki/Radius_of_gyration) (Rg), which is a measure of chain compactness.

Let's plot a the distribution of the radius of gyration of the conformations in our ensemble.

❓**Question**❓: given an IDPs and a globular (folded) protein with the same number of amino acids, which one do you expect to have an higher Rg?

In [ ]:
vis.radius_of_gyration(violin_plot=True);

Nice! But right now this is not very informative... Rg mostly makes sense in relative terms (e.g.: the Rg of protein X is larger/smaller the one of protein Y).

Later, you will generate ensembles for many IDPs and will study the relationship of Rg with sequence length.

Hopefully you will get a better intution of Rg.

Let's also plot the [contact map](https://en.wikipedia.org/wiki/Protein_contact_map) of the ensemble, another typical analysis tool of IDP ensembles.

**Contact maps**: they indicate the probability of two residues (beads) to have a distance below a threshold (8.0 Angstrom is the default in IDPET) in the ensemble.

In [ ]:
vis.contact_prob_maps(
    avoid_zero_count=True,
    log_scale=True,
    color='Oranges',
    threshold=0.8  # the threshold is expressed in units of nanometers.
);

❓**Question**❓: why are contact frequencies between residues that are far apart in sequence decreasing in our IDP? Is this something you would expect in a typical folded protein structure?

# Part 3/6 - Multi-ensemble comparison

Let's compare our idpGAN protac ensemble to other ensembles:
* **A protac ensemble from COCOMO-1**: idpGAN was trained to replicate COCOMO-1. Let's analyze the protac ensemble from actual COCOMO-1 simulations to check if the model is actually approximating MD.
* **poly-ala ensemble**: in COCOMO-1, every protein tends to behave as an IDP. However, different sequences have result in different ensemble properties. Let's use the MD ensemble from a poly-alanine chain of the same length of protac to check if the model can capture sequence-specific behavior.

Let's begin a new IDPET analysis with three ensembles.

In [ ]:
from idpet.ensemble import Ensemble
from idpet.ensemble_analysis import EnsembleAnalysis
from idpet.visualization import Visualization


# Load the ensemble data (,  and
seq_names = [
    "idpgan.protac",  # idpGAN ensemble of protac.
    "protac",         # COCOMO1 ensemble of protac.
    "polyala"         # COCOMO1 ensemble for a poly-alanine sequence of the
                      # same length of protac.
]

ensembles = []
for seq_name_i in seq_names:
    ensemble = Ensemble(
        code=seq_name_i,
        data_path=os.path.join(data_path, f"{seq_name_i}.xtc"),
        top_path=os.path.join(data_path, f"{seq_name_i}.pdb")
    )
    ensembles.append(ensemble)

analysis_multi = EnsembleAnalysis(ensembles)
analysis_multi.load_trajectories()
vis_multi = Visualization(analysis=analysis_multi)

First, let's plot some statistics about our new ensembles. This is useful to quickly see how many residues and conformations our proteins have.

In [ ]:
analysis_multi.get_features_summary_dataframe(
    selected_features=["rg", "end_to_end", "flory_exponent"]
)

## IDPET compactness and contact plots

Let's visualize Rg distributions.

In [ ]:
vis_multi.radius_of_gyration(violin_plot=True);

Let's also visualize the histograms of the end-to-end distances (the distance between the N-term. and C-term. beads of conformations). This is also a polular property analyzed in IDP ensemble analyses.

In [ ]:
vis_multi.end_to_end_distances(violin_plot=True, summary_stat='mean', rg_norm=False);

Let's also plot the contact maps of the ensembles.

In [ ]:
vis_multi.contact_prob_maps(
    avoid_zero_count=True,
    log_scale=True,
    color='Blues',
    threshold=0.8  # the threshold is expressed in units of nanometers.
);

What do you think about this comparison so far?

* ❓**Question 1**❓: is the idpGAN ensemble closely emulating the reference MD ensemble?
* ❓**Question 2**❓: are the protac and poly-ala contact maps consistent with the corresponding Rg violin plots? Hint: look at the contact frequencies between the N- and C-term of the proteins.
* ❓**Question 3 (important)**❓: why are the contacts maps of polyala and protac so different? Hint: look at the sequence of protac and try to find a physiochemical explanation.

## Visualizing 3D structures

Finally, let's actually view some 3D structures of these ensembles.

Colab does not easily allow to use advanced molecular visualization software. We will use [Py3DMol](https://github.com/avirshup/py3dmol), which is still quite good.

Visualizing the 3D structures of your ensembles is crucial in ensemble analysis!

**How**: interact with the mouse with the 3d molecule below.

In [ ]:
# change here to visualize other ensembles.
ensemble_name = "idpgan.protac"
# NOTE: change here to visualize another conformation.
# set to `conformation = None`, to view different conformations
# in an animation.
conformation_index = 0


sel_ensemble = analysis_multi[ensemble_name]
sel_rg = sel_ensemble.get_features("rg")
ee_distances = sel_ensemble.get_features("end_to_end")
if conformation_index is not None:
    conformation = sel_ensemble.trajectory[conformation_index]
    print(f"Rg of conformation {conformation_index}: {sel_rg[conformation_index]:.3f} nm")
    print(f"End-to-end distance of conformation {conformation_index}: {ee_distances[conformation_index]:.3f} nm")
    print('Blue is N-term. and red is C-term.')
else:
    conformation = sel_ensemble.trajectory

view = show_structure(
    snapshot=conformation,
    representation='cg',
    cmap_name='Spectral_r'
)
view.show()

❓**Question**❓: are Rg and end-to-end distances correlated? (only for experienced coders: can you make a plot of Rg vs end-to-end of a few conformations?). You can get the two arrays with:
```python
sel_rg = sel_ensemble.get_features("rg")
ee_distances = sel_ensemble.get_features("end_to_end")
```
and plot them with `plt.scatter(array_1, array_2)` function of matplotlib.

# Part 4/6 - Generate ensembles for your own sequence

Before moving to the final two parts, you may go back to Part 3 and generate an ensemble for **any sequence of your choice**.

You will not have a reference MD ensemble for this sequence, the goal is to analyze the ensemble on its own.

Generate the ensemble and use the IDPET visualization tools from Part 3 (e.g., `vis.contact_prob_maps`) to characterize it.

This section is meant for exploration. The goal is to develop intuition and test the limits of generative models.

If you need inspiration, here are some ideas:

### 🧪 Physiochemically weird sequences of the same length of protac

- Try **poly-TRP** (very hydrophobic)  
- Try **poly-GLU** or **poly-LYS** (highly charged chains)  
- Try sequence patches (e.g., first 50% of the sequence is GLU, last 50% LYS)

Before generating the ensemble, pause and predict:

- What should Rg look like with respect to the wt protac?
- What should the contact map look like?
- Does the generated ensemble match your physical intuition?

### 🧬 A globular protein sequence

Go to UniProt and retrieve the sequence of a globular protein that you know.

- Can idpGAN fold it?
- What do the generated structures look like?

### 📝 3. Introduce mutations to edit the sequence

Get the protac sequence and start adding groups of 3-4 point mutations (you can decide which residue to mutate or do it randomly). Repeat for 2-3 cycles.

- How many mutations you need to make the ensemble properties significantly different from the ones of the wt sequence?
- Can you design an IDP sequence with some desired property (e.g.: strong contacts between residue X and Y)?
- **Hint**: if you want to compare plots of different mutants, you can save the plots as files by right clicking on them and download the images.

# Part 5/6 - Testing the limits of idpGAN in predicting experimental behavior

Looks like that idpGAN can emulate COCOMO-1 behavior, at least under certain conditions.

COCOMO-1, was parametrized to reproduce the experimental Rg of IDPs. So idpGAN can be used to obtain ensembles with Rg values that are in reasonable agreement to experimental ones.

But is this always true? We need to understand when/how idpGAN fails.

## Experimental radius-of-gyration data

To find out idpGAN limits, let's generate ensembles for a series of IDPs with experimentally-measured Rg values (kindly provided by Giulio Tesei). These values were obtained by techniques like SAXS or FRET.

Here we will use a random subset of this data.

In [ ]:
rg_data = [
 {'name': 'Nup49_Rg',
  'exp_rg': 1.6,
  'seq': 'GCQTSRGLFGNNNTNNINNSSSGMNNASAGLFGSKPFA'},
 {'name': 'DomainV',
  'exp_rg': 2.43,
  'seq': 'GAMGTAGNKAALLDQIREGAQLKKVEQNSRPVSCSGRDALLDQIRQGIQLKSVADGQESTPPTPAPT'},
 {'name': 'DSS1',
  'exp_rg': 2.5,
  'seq': 'MSRAALPSLENLEDDDEFEDFATENWPMKDTELDTGDDTLWENNWDDEDIGDDDFSVQLQAELKKKGVAAC'},
 {'name': 'HeV_PNT3_CTD_3Y3A',
  'exp_rg': 2.73,
  'seq': 'MSYYHHHHHHLESTSLYKKAGFTPTEEPPVIPEAAAGSGRRGDLSKSPPRGNVNLDSIKIYTSDDEDENQLEYEDEF'},
 {'name': 'NUS_Rg',
  'exp_rg': 2.5,
  'seq': 'GCPSASPAFGANQTPTFGQSQGASQPNPPGFGSISSSTALFPTGSQPAPPTFGTVSSSSQPPVFGQQPSQSAFGSGTTPNFA'},
 {'name': 'SMAD2_linker',
  'exp_rg': 2.99,
  'seq': 'GPLPPVLVPRHTEILTELPPLDDYTHSIPENTNFPAGIEPQSNYIPETPPPGYISEDGETSDQQLNQSMDTGSPAELSPTTLSPVNHSLD'},
 {'name': 'VWF',
  'exp_rg': 3.08,
  'seq': 'MEDREQAPNLVYMVTGNPASDEIKRLPGDIQVVPIGVGPNANVQELERIGWPNAPILIQDFETLPREAPDLVLQRGGWSHPQFEKGGGSGGGSGGWSHPQFEK'},
 {'name': 'p27Cv56',
  'exp_rg': 2.328,
  'seq': 'GSHMKGACGSSVLGTGNPRNQAHVSDTSLEEDDDEQDDSTPDEVSQACTIVASALDINAATPRSPKASPKRKRKRQSTAPAQGNEPPGNAGSVEQTPKKPGLRRRQT'},
 {'name': 'lambdaN',
  'exp_rg': 3.8,
  'seq': 'MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKSRVESALNPIDLTVLAEYHKQIESNLQRIERKNQRTWYSKPGERGITCSGRQKIKGKSIPLI'},
 {'name': 'NUL_Rg',
  'exp_rg': 3.0,
  'seq': 'GCGFKGFDTSSSSSNSAASSSFKFGVSSSSSGPSQTLTSTGNFKFGDQGGFKIGVSSDSGSINPMSEGFKFSKPIGDFKFGVSSESKPEEVKKDSKNDNFKFGLSSGLSNPVFA'},
 {'name': 'NHE6cmdd',
  'exp_rg': 3.2,
  'seq': 'GPPLTTTLPACCGPIARCLTSPQAYENQEQLKDDDSDLILNDGDISLTYGDSTVNTEPATSSAPRRFMGNSSEDALDRELAFGDHELVIRGTRLVLPMDDSEPPLNLLDNTRHGPA'},
 {'name': 'P7R',
  'exp_rg': 2.71,
  'seq': 'GSMASASSSQRGRSGRGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGRYGGSGDRYNGFGNDGRNFGGGGSYNDFGNYNNQSSNFGPMKGGNFRGRSSGPYGRGGQYFAKPRNQGGYGGSSSSRSYGSGRRF'},
 {'name': 'aSyn140',
  'exp_rg': 3.55,
  'seq': 'MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA'},
 {'name': 'HvASR1',
  'exp_rg': 3.46,
  'seq': 'GSPEFMAEEKHHHHLFHHKKEGEDFQPAADGGADVYGYSTETVVTGTGNEGEYERITKEEKHHKHKEHLGEMGAVAAGAFALYEKHEAKKDPEHAHKHKIEEELAAAAAVGAGGFVFHEHHEKKQDHKEAKEASGEKKHHLFG'},
 {'name': 'ANAC046',
  'exp_rg': 3.6,
  'seq': 'NAPSTTITTTKQLSRIDSLDNIDHLLDFSSLPPLIDPGFLGQPGPSFSGARQQHDLKPVLHHPTTAPVDNTYLPTQALNFPYHSVHNSGSDFGYGAGSGNNNKGMIKLEHSLVSVSQETGLSSDVNTTATPEISSYPMMMNPAMMDGSKSACDGLDDLIFWEDLYTS'},
 {'name': 'ERNTD_S118D',
  'exp_rg': 3.69,
  'seq': 'MTMTLHTKASGMALLHQIQGNELEPLNRPQLKIPLERPLGEVYLDSSKPAVYNYPEGAAYEFNAAAAANAQVYGQTGLPYGPGSEAAAFGSNGIGGFPPLNSVSPSPLMLLHPPPQLDPFLQPHGQQVPYYLENEPSGYTVREAGPPAFYRPNSDNRRQGGRERLASTNDKGSMAMESAKETRY'},
 {'name': 'CAHSD',
  'exp_rg': 4.84,
  'seq': 'MSGRNVESHMERNEKVVVNNSGHADVKKQQQQVEHTEFTHTEVKAPLIHPAPPIISTGAAGLAEEIVGQGFTASAARISGGTAEVHLQPSAAMTEEARRDQERYRQEQESIAKQQEREMEKKTEAYRKTAEAEAEKIRKELEKQHARDVEFRKDLIESTIDRQKREVDLEAKMAKRELDREGQLAKEALERSRLATNVEVNFDSAAGHTVSGGTTVSTSDKMEIKRN'},
 {'name': 'K44',
  'exp_rg': 5.2,
  'seq': 'MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKAEEAGIGDTPSLEDEAAGHVTQARMVSKSKDGTGSDDKKAKGADGKTKIATPRGAAPPGQKGQANATRIPAKTPPAPKTPPSSGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIE'},
 {'name': 'GHRICD',
  'exp_rg': 6.0,
  'seq': 'SKQQRIKMLILPPVPVPKIKGIDPDLLKEGKLEEVNTILAIHDSYKPEFHSDDSWVEFIELDIDEPDEKTEESDTDRLLSSDHEKSHSNLGVKDGDSGRTSCCEPDILETDFNANDIHEGTSEVAQPQRLKGEADLLCLDQKNQNNSPYHDACPATQQPSVIQAEKNKPQPLPTEGAESTHQAAHIQLSNPSSLSNIDFYAQVSDITPAGSVVLSPGQKNKAGMSQCDMHPEMVSLCQENFLMDNAYFCEADAKKCIPVAPHIKVESHIQPSLNQEDIYITTESLTTAAGRPGTGEHVPGSEMPVPDYTSIHIVQSPQGLILNATALPLPDKEFLSSCGYVSTDQLNKIMP'},
 {'name': 'ht2N4R',
  'exp_rg': 6.7,
  'seq': 'MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTPTAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQARMVSKSKDGTGSDDKKAKGADGKTKIATPRGAAPPGQKGQANATRIPAKTPPAPKTPPSSGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKDNIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSASLAKQGL'}
]

## Generate the ensembles

Let's start generating idpGAN ensembles for these sequences.


In [ ]:
# Helper function, feel free to use. Allows you to compute the Rg of
# idpGAN conformations directly on NumPy arrays. In this way, you don't
# need to convert them to trajectories and analyze in IDPET.


def compute_rg(xyz):
    """
    Takes as input a numpy array of shape (B, L, 3).
        B: batch size, L: number of atoms.
    Adapted from the mdtraj library: https://github.com/mdtraj/mdtraj.
    """
    num_atoms = xyz.shape[1]
    # Assign equal masses to each residue, a tolerable approximation here.
    masses = np.ones(num_atoms)
    weights = masses / masses.sum()
    mu = xyz.mean(1)
    centered = (xyz.transpose((1, 0, 2)) - mu).transpose((1, 0, 2))
    squared_dists = (centered ** 2).sum(2)
    Rg = (squared_dists * weights).sum(1) ** 0.5
    return Rg

❗️️️️️**Important note**❗️: Note that we generally consider experimental values as ensemble-average values. Therefore, when you generate/simulate an ensemble, you should compare its average Rg value with the experimental one.

In [ ]:
# Will use less conformation here to speed up things.
num_conformations_rg = 100
idpgan_ensembles = []
idpgan_rg = []

for s in rg_data:
    print(f"sampling {s['name']} L={len(s['seq'])}")
    xyz_gen_i = netg.predict_idp(
        n_samples=num_conformations_rg, aa_seq=s["seq"],
        device=device
    ).cpu().numpy()
    idpgan_ensembles.append(xyz_gen_i)
    rg_gen_i = compute_rg(xyz_gen_i)
    avg_rg_gen_i = rg_gen_i.mean()
    idpgan_rg.append(avg_rg_gen_i)






❓**Question (important)**❓: why are experimentally-measured values considered ensemble-averages? They are averages over what?

❓**Question (optional)**❓: have you noticed how the speed of sampling is changing with sequence length? Can you generate the ensembles again and print (or plot, if you want) the "sequence length " vs "ensemble generation speed" relationship? You can time a Python function in your code via:
```python
import time

time_start = time.time()
your_python_func()  # call your function here.
time_end = time.time()
time_it_took = time_end - time_start  # by default, in units of seconds.
print(time_it_took)
```

Is the relationship linear? If not, where might be this coming from (hint: the idpGAN generator is a transformer neural network that has sequences of length `N` as input/output. Try googling "transformers computational complexity"). What is the neural network architecture of more modern deep ensemble generators (e.g.: BioEmu)? Do you expect similar "protein size" vs "running time" relationships?

## Comparison with experiments

Now, let's plot the "predictions" and "experimental values" vs "length".

In [ ]:
idp_lengths = [len(r["seq"]) for r in rg_data]
exp_rg = [r["exp_rg"] for r in rg_data]

plt.scatter(idp_lengths, exp_rg, label="experimental")
plt.scatter(idp_lengths, idpgan_rg, label="idpGAN")
plt.xlabel("length")
plt.ylabel("Rg [nm]")
plt.legend()
plt.show()

❓**Question:**❓ are predictions equally good across the length scale? Or do they get worse in some condition?

## Checking the physical realism of idpGAN conformations

From literature ([COCOMO-1 paper](https://pubs.acs.org/doi/10.1021/acs.jctc.2c00856)) we know that COCOMO-1 tends to overestimate the compactness of longer IDPs. In the paper they wrote:
```
"... our model also shows systematic deviations where smaller systems tend to be less compact, whereas larger systems are more compact than suggested by the experimental values."
```

Ok! Is our observation explainable only by the properties of the COCOMO-1 training data? (Indeed: this over-compactness issue was solved in [COCOMO-2](https://pubs.acs.org/doi/10.1021/acs.jctc.4c01460))!

idpGAN is just emulating its training data and everything is explained, right?

Maybe, but we are interested in truly understanding the behavior our model! Let's investigate further.

### Inspection of idpGAN conformations

One of the weakness of current AI ensemble/structure generators is the physical realism of their conformations: sometimes they contain atom-atom clashes or bond lenth violations, which is quite bad.

Let's define some physical errors in COCOMO-1 CG structures:

* **Clashes**: any pair of non-bonded (i.e.: non adjacent) beads at a distance: $d < 0.25\,nm$ (note that a bead in COCOMO-1 represents a residue, so 0.30 nm is a very short distance for two residues).
* **Bond length violations**: any pair of bonded (i.e.: adjacent along the primary sequence) beads with a distance outside the range: $0.30\,nm < d < 0.46\,nm$.

Do idpGAN conformations contain such inaccuracies? Below we define some functions that can be used to evaluate these structural inaccuracies.

In [ ]:
#
# NOTE: this functions are only valid for CG protein structures. They will not
# work with all-atom protein structures (in the last part of the notebook).
#

def compute_bond_lengths(xyz):
    """Using numpy vectorization."""
    # distance vectors between all pairs of adjacent residues. shape: (B, L-1, 3)
    delta = xyz[:,:-1,:] - xyz[:,1:,:]
    # distances between all pairs of adjacent residues. shape: (B, L-1)
    dist = np.sqrt(
        # squared norm of the distance vectors.
        np.sum(
            np.square(delta), axis=-1
        )
    )
    return dist

def compute_bond_length_violations_in_ensemble(bond_lengths, min_d=0.30, max_d=0.46):
    """Compute the average number of violations in an ensemble"""
    # boolean array for labeling violations. shape: (B, L-1)
    violations = (min_d > bond_lengths) | (bond_lengths > max_d)
    # compute the number of violations of each conformation. shape: (B, )
    violations = violations.sum(axis=1)
    # compute the average over all conformations.
    violations = violations.mean(axis=0)
    return violations


def compute_nonbonded_distances(xyz):
    """Using numpy vectorization."""
    # Get the indices of all pair of non-bonded atoms. shape: [2, num_distances]
    indices = np.triu_indices(
        xyz.shape[1],
        k=2  # exclude pairs that have sequence separation < 2.
    )
    # distance vectors between all pairs of non-bonded residues. shape: (B, num_distances, 3)
    delta = xyz[:,indices[0],:] - xyz[:,indices[1],:]
    # distances between all pairs of adjacent residues. shape: (B, L-1)
    dist = np.sqrt(
        # squared norm of the distance vectors.
        np.sum(
            np.square(delta), axis=-1
        )
    )
    return dist

def compute_clashes_in_ensemble(nonbonded_distances, min_d=0.25):
    """Compute the average number of clashes in an ensemble"""
    # boolean array for labeling violations. shape: (B, L-1)
    violations = (min_d < nonbonded_distances)
    # compute the number of clashes of each conformation. shape: (B, )
    violations = violations.sum(axis=1)
    # compute the average over all conformations.
    violations = violations.mean(axis=0)
    return violations

Let's inspect the 3d structures of some idpGAN conformations:
- Let's first analyze conformations for a protein that idpGAN seems to model reasonably well (Nup49_Rg).
- Then, find the name of a protein that idpGAN models incorrectly and visualize its conformations.

❓**Questions**❓: Can you spot some differences in physical properties (e.g.: clashes)? Also, use the histograms below.

In [ ]:
# change here to visualize another protein.
ensemble_name = "Nup49_Rg"
# change here to visualize another conformation.
# set to `conformation = None`, to view different conformations
# in an animation.
conformation_index = 0

sel_idx = None
for i, r in enumerate(rg_data):
    if r["name"] == ensemble_name:
        sel_idx = i
if sel_idx is None:
    raise IndexError(f"No ensemble with name {ensemble_name} was found")
sel_rg_data = rg_data[sel_idx]
sel_xyz = idpgan_ensembles[sel_idx]
conformation = xyz_to_traj(sel_xyz, sel_rg_data["seq"])

if conformation_index is not None:
    conformation = conformation[conformation_index]

print('Blue is N-term. and red is C-term.')

view = show_structure(
    snapshot=conformation,
    representation='cg',
    cmap_name='Spectral_r'
)
view.show()

Can you spot outliers that violate the thresholds in the histograms below?

In [ ]:
bond_lengths = compute_bond_lengths(sel_xyz).ravel()

plt.title(f"bond lengths for {ensemble_name}")
plt.xlabel("bond length [nm]")
plt.ylabel("density")
plt.hist(bond_lengths, bins=50)
plt.axvline(x=0.30, color="C3", label="thresholds")
plt.axvline(x=0.46, color="C3")
plt.legend()
plt.show()

In [ ]:
nonbonded_distances = compute_nonbonded_distances(sel_xyz).ravel()

plt.title(f"non-bonded distances for {ensemble_name}")
plt.xlabel("distance [nm]")
plt.ylabel("density")
plt.hist(nonbonded_distances, bins=50)
plt.axvline(x=0.25, color="C3", label="threshold")
plt.legend()
plt.show()

### Sequence length vs physical inaccuracies

Finally, compute our scores for **all proteins** and plot our results as a function of sequence length.

️❗️️**Note**️❗️: there are some missing parts in this code! Can you fill-in the gaps?

In [ ]:
idpgan_bond_length_viol = []
idpgan_clashes = []

for i in range(len(rg_data)):

    print(f"analyzing {rg_data[i]['name']} L={len(rg_data[i]['seq'])}")

    xyz = idpgan_ensembles[i]
    bond_lengths = None  # MISSING PART: Use the correct function here.
    bond_length_violations = None  # MISSING PART: Use the correct function here.

    nonbonded_distances = None  # MISSING PART: Use the correct function here.
    clashes = None  # MISSING PART: Use the correct function here.

    if any(
        [
            a is None for a in \
            [bond_lengths, bond_length_violations, nonbonded_distances, clashes]
        ]
    ):
        raise ValueError("You need to add all the correct function from the cell above!")

    idpgan_bond_length_viol.append(bond_length_violations)
    idpgan_clashes.append(clashes)

Plot the "length" vs "bond length violations"

In [ ]:
idp_lengths = [len(r["seq"]) for r in rg_data]
plt.scatter(idp_lengths, idpgan_bond_length_viol)
plt.xlabel("length")
plt.ylabel("avg. of bond length violations in a conformation")
plt.show()

And finally the clashes.

In [ ]:
idp_lengths = [len(r["seq"]) for r in rg_data]

plt.scatter(idp_lengths, idpgan_clashes)
plt.xlabel("length")
plt.ylabel("ensemble_average[clashes in a conformation]")
plt.show()

❓**Question**❓: what is your final assessment? Is physical quality deteriorating with increasing lengths?

## Conclusions

Congrats! You broke idpGAN and have correctly diagnosed limits of a deep generative model.

An MD force field has strongly repulsive forces for non-bonded atoms and a strong bond length potential, it could never results in such unphysical conformations!

Why is idpGAN doing such a poor job on IDPs longer than ~100 residues? By looking at the [idpGAN paper](https://www.nature.com/articles/s41467-023-36443-x) you can see the maximum effective length was 120 residues. This explains a lot. It has never seen proteins longer than this.

Also, its neural network and training procedure were not optimal (with a small neural network and an heuristic GAN training procedure for handling variable sequence lengths).

Therefore the extrapolation ability of the model is limited.

Modern ensemble generators have more robust training and extrapolation abilities, with vastly improved physical realism (see below).

But they may also have limits related to training data (e.g.: length, disorder fraction, sequence features, domain architecture, etc...). For this reason, always analyze your data carefully!

# Part 6/6 - Atomistic ensembles and last-generation deep ensemble generators

## More recent ML ensemble generators

Finally, we are going to analyze some atomistic IDP ensembles from more recent ML ensemble generator.

The goal is to introduce you to the analysis of atomistic IDP ensembles and compare modern ML generators to idpGAN.

Unfortunately, running these modern methods is computationally expensive and cannot be easily done in a single Colab notebook. You will have to download pre-generated ensembles from our GitHub repo.

## Test case
We will analyze different ensembles of [ACTR](https://pubs.acs.org/doi/10.1021/ja4045532) a 69-residue IDP (it is often found in IDP ensemble benchmarks).

## Ensemble generation methods

Here is the list of ensembles we will analyze:
- **Reweighted atomistic MD (PED00536e001)**: this is an ensemble deposited on [PED](https://proteinensemble.org/entries/PED00536). It was recently constructed by *Borthakur et al.* by running [extensive MD simulations followed by reweighting against experimental data](https://www.nature.com/articles/s41467-025-64098-3#ref-CR53). It was found to be in excellent agreement with the experimental data (NMR and SAXS), we can consider it an high-quality ensemble.
- **idpGAN (idpgan-aa)**: an idpGAN ensemble, all-atom details were reconstructed in a post-processing step using the [cg2all neural network](https://www.sciencedirect.com/science/article/pii/S096921262300401X).
- **BioEmu (bioemu)**: an ensemble generated by the [BioEmu](https://www.science.org/doi/10.1126/science.adv9817) AI ensemble generator, using its default parameters. BioEmu was not trained on IDPs, but [was reported](https://www.biorxiv.org/content/10.1101/2025.10.18.680935v3) to generate IDP ensembles that are in somewhat good agreement with experimental data.
- **Boltz-2 default (boltz2)**: ensemble generated by running [Boltz-2](https://www.biorxiv.org/content/10.1101/2025.06.14.659707v1) with default parameters, using as input a multiple sequence alignment (MSA). Boltz-2 has not been trained on IDPs. Running Boltz-2 in this modality [reportedly](https://www.biorxiv.org/content/10.1101/2025.10.18.680935v3) does not produce IDP ensembles with good agreement with experimental data.

## Goal

We know that the PED00536e001 ensemble is in good agreement with experimental data. You goal is to see if the ensembles generated by AI methods can recapitulate its main structural properties.

In [ ]:
#@title Download the ML-generated ensembles

#@markdown Run this cell to download the ensembles

data_path = "data"

# download PED ensemble. The original ensemble contains a few extra-atoms, we remove them here.
ped_code = "PED00536"
!wget https://deposition.proteinensemble.org/api/v1/entries/{ped_code}/ensembles/e001/ensemble-pdb
!tar -xzvf ensemble-pdb
!rm ensemble-pdb
with open("filter.txt", "w") as o_fh:
    o_fh.write("""ATOM   1069  H   GLY A   1
ATOM   1070  HE2 HIS A  36
ATOM   1071  N   GLY A   1
ATOM   1072  H   GLY A   1""")
!grep -v -f filter.txt pdbfile.pdb | sed '/^MODEL *250/,$d' > output.pdb
!rm pdbfile.pdb
_traj = mdtraj.load("output.pdb")
_traj.save(f"{ped_code}e001.xtc")
_traj[0].save(f"{ped_code}e001.pdb")
!cp {ped_code}e001.* {data_path}/
!rm output.pdb

# idpGAN ensembles
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/idpgan.aa.actr.pdb -P $data_path
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/idpgan.aa.actr.xtc -P $data_path
# BioEmu
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/bioemu.dpm.pdb -P $data_path
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/bioemu.dpm.xtc -P $data_path
# Boltz-2
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/boltz2.msa.pdb -P $data_path
!wget https://github.com/giacomo-janson/ml4ngp_training_school_2026/raw/refs/heads/main/data/ensembles/boltz2.msa.xtc -P $data_path


## Initialize the IDPET analysis

Run the cell below to initialize the analysis.

**Note**: IDPET will have to download and setup the PED00536e001 ensemble, which might take ~1-2 minutes.

In [ ]:
from idpet.ensemble import Ensemble
from idpet.ensemble_analysis import EnsembleAnalysis
from idpet.visualization import Visualization


aa_ensembles = [
    # This ensemble that can be downloaded from PED via IDPET.
    Ensemble(
        code='PED00536e001',
        data_path=os.path.join(data_path, f"PED00536e001.xtc"),
        top_path=os.path.join(data_path, f"PED00536e001.pdb")
    ),

    # We will load the other ensembles by loading their topology+trajectory files
    # that we downloaded above.
    Ensemble(
        code="idpgan-aa",
        data_path=os.path.join(data_path, f"idpgan.aa.actr.xtc"),
        top_path=os.path.join(data_path, f"idpgan.aa.actr.pdb")
    ),
    Ensemble(
        code="bioemu",
        data_path=os.path.join(data_path, f"bioemu.dpm.xtc"),
        top_path=os.path.join(data_path, f"bioemu.dpm.pdb")
    ),
    Ensemble(
        code="boltz2",
        data_path=os.path.join(data_path, f"boltz2.msa.xtc"),
        top_path=os.path.join(data_path, f"boltz2.msa.pdb")
    )

]
aa_analysis = EnsembleAnalysis(
    aa_ensembles,
)
aa_analysis.load_trajectories();
aa_analysis.random_sample_trajectories(sample_size=125);
aa_vis = Visualization(analysis=aa_analysis)

In [ ]:
aa_analysis.get_features_summary_dataframe(
    selected_features=["rg", "end_to_end", "flory_exponent"]
)

## Compactness and contact analysis (global features)

Let's first visualize the Rg of the ensembles.

In [ ]:
actr_experimental_rg = 2.63

ax = aa_vis.radius_of_gyration(violin_plot=True)
ax.axhline(y=actr_experimental_rg,
           linestyle="--", color="gray",
           label="experimental value (SAXS)")
ax.legend()
plt.show()

Let's also plot the contact maps of the ensembles.

In [ ]:
aa_vis.contact_prob_maps(
    avoid_zero_count=True,
    log_scale=True,
    color='Blues',
    threshold=0.8  # the threshold is expressed in units of nanometers.
);

❓**Questions**❓:
- Based on the Rg distribution and contact map patterns, are the ensembles similar to each other?
- In the Rg plot, we also included the [experimental Rg value for this protein](https://onlinelibrary.wiley.com/doi/abs/10.1002/pro.435). Which ensemble appears to be in best agreement?

## Secondary structure analysis (local features)

According to experiment, ACTR is reported to form partially stable alpha helices. Are there any helices in our ensembles?

The plot below reports the frequency of a residue to be in helical state in an ensemble (0: never helix; 1: always helix).

In [ ]:
aa_vis.relative_dssp_content(
    dssp_code='H',  # H: analyze helicity.
    figsize=(6,3)
);

The PED00536e001 ensemble was reported to have excellent agreement with NMR data that allows to quantify backbone torsion preference experimentally. So, its helicity is probably a good representation of the experimental behavior.

❓**Questions**❓:
- how are the deep generative models capturing the helicity? Are they capturing the correct helical regions and helix abundance?
- How is idpGAN doing with helicity? Is it producing any? If not, why?

## Structural analysis exploration

Let's visualize some actual 3D structures. At a first glance, do the AI-generated structures look like the MD ones from PED00536e001?

In [ ]:
print("- Use the following codes below:", aa_analysis.ens_codes)

In [ ]:
ensemble_name = "PED00536e001"      # change here to visualize other ensembles.
conformation_index = 0             # change here to visualize another conformation.

sel_ensemble = aa_analysis[ensemble_name]
if conformation_index is not None:
    conformation = sel_ensemble.trajectory[conformation_index]
    sel_rg = sel_ensemble.get_features("rg")
    ee_distances = sel_ensemble.get_features("end_to_end")
    print(f"Rg of conformation {conformation_index}: {sel_rg[conformation_index]:.3f} nm")
    print(f"End-to-end distance of conformation {conformation_index}: {ee_distances[conformation_index]:.3f} nm")
else:
    conformation = sel_ensemble.trajectory

print('Blue is N-term. and red is C-term.')

view = show_structure(
    snapshot=conformation,
    representation='aa',
    cmap_name='Spectral_r'
)
view.show()

❓**Question**❓:
- do BioEmu structures contain all atoms? Or are they missing some? Here is the ACTR sequence: `GTQNRPLLRNSLDDLVGPPSNLEGQSDERALLDQLHTLLSNTDATGLEEIDRALGIPELVNQGQALEPKQD`.
- is idpGAN showing any atomic clashes?
- how do the Boltz-2 conformations (which have a low Rg) look like?

## Conclusions
A complete evaluate of these ensembles will involve comparison with extensive experimental data (SAXS, many NMR observables), which is beyond the scope of this notebook.

However, based on your analysis:
- Which ensemble better captures the experimental Rg?
- Which ML ensemble better captures the helicity observed in the MD ensemble?
- Is there an ML ensemble which fully captures Rg and helicity at the same time?
- Can you rationalize the properties of these ML ensembles based on their training data?